In [2]:
import random
import numpy as np
import networkx as nx 
from dijkstar import Graph, find_path

In [101]:
def generate_instance_matrix(nb_vertex, min_weight, max_weight, density):
  graph = np.ones((nb_vertex, nb_vertex)) * np.inf
  for i in range(nb_vertex - 1):
    for j in range(i+1, nb_vertex):
      if random.uniform(0, 1) < density:
        weight = random.randint(min_weight, max_weight)
        graph[i, j] = weight
        graph[j, i] = weight
  for i,line in enumerate(graph):
    index_zeros = np.where(line == np.inf)[0]
    index_zeros = np.delete(index_zeros, np.where(index_zeros == i))
    for j in range(len(index_zeros) - (nb_vertex - 3)):
      index = random.choice(index_zeros)
      index_zeros = np.delete(index_zeros, np.where(index_zeros == index))
      weight = random.randint(min_weight, max_weight)
      graph[i, index] = weight
      graph[index, i] = weight
  return graph

def convert_graph(graph):
  graph_dijkstra = Graph()
  for i in range(len(graph)):
    for j in range(len(graph)):
      graph_dijkstra.add_edge(i, j,graph[i][j])
  return graph_dijkstra

def get_distance_between_nodes(graph, i, j):
  if (graph[i][j] != np.inf):
    return graph[i][j], [i, j]
  else:
    if i != j:
      shortest_path = find_path(graph, i,j)
      return shortest_path.total_cost, shortest_path.nodes
    else:
      return 0, None

def get_complete_graph(graph, node_list):
  all_paths = {}
  converted_graph = convert_graph(graph)
  complete_graph = np.zeros((len(node_list), len(node_list)))
  for i in node_list:
    voisins_array = np.delete(node_list, np.where(node_list == i))
    for j in voisins_array:
      distance, path = get_distance_between_nodes(converted_graph, i, j)
      converted_graph_i = np.where(node_list == i)[0][0]
      converted_graph_j = np.where(node_list == j)[0][0]
      complete_graph[converted_graph_i][converted_graph_j] = distance
      all_paths[(converted_graph_i, converted_graph_j)] = path
  return complete_graph, all_paths

def get_random_vertex(size, nb_vertex):
  random_vertex = []
  all_vertex = np.array(range(size))
  for _ in range(nb_vertex):
    value = random.choice(all_vertex)
    random_vertex.append(value)
    all_vertex = np.delete(all_vertex, np.where(all_vertex == value))
  return np.array(random_vertex)

def generate_interval():
  interval = random.choice(range(1,4))
  shift = random.choice(range(0, 11-interval))
  return(shift*60, (shift+interval)*60)

def generate_instance(nb_vertex_matrix, min_weight, max_weight, density, random_vertex):
  graph = generate_instance_matrix(nb_vertex_matrix, min_weight, max_weight, density)
  complete_graph, all_paths = get_complete_graph(graph, random_vertex)
  return graph, complete_graph, all_paths

def get_complete_path(path, all_paths):
  complete_path = []
  for i in range(len(path) - 1):
    complete_path.extend(all_paths[(path[i], path[i+1])][:-1])
    if i == len(path) - 2:
      complete_path.append(all_paths[(path[i], path[i+1])][-1])
  return complete_path

def is_path_in_interval(path, complete_graph, interval, vertex):
  weight = 0
  for i in range(np.where(np.array(path) == vertex)[0][0]):
    weight += complete_graph[path[i]][path[i+1]]
  if weight >= interval[0] and weight <= interval[1]:
    return True
  else:
    return False

def generate_neighbours(path, complete_graph, interval, vertex):
  neighbours = []
  for i in range(len(path)):
    for j in range(i + 1, len(path)):
      new_neighbour = path.copy()
      new_neighbour[i] = path[j]
      new_neighbour[j] = path[i]
      if is_path_in_interval(new_neighbour, complete_graph, interval, vertex):
        neighbours.append(new_neighbour)
  return neighbours

def generate_random_path(nb_vertex):
  path = np.arange(nb_vertex)
  np.random.shuffle(path)
  return path

In [102]:
# variables
nb_vertex_matrix = 100
min_weight = 10
max_weight = 60
density = 0.3
nb_vertex_chosen = 20

In [106]:
#random generation of a list of vertex among all the vertex available
random_vertex = get_random_vertex(nb_vertex_matrix, nb_vertex_chosen)

# generation of the graph, the complete graph and the list of all paths
graph, complete_graph, all_paths =\
generate_instance(
  nb_vertex_matrix = nb_vertex_matrix,
  min_weight = min_weight,
  max_weight = max_weight,
  random_vertex = random_vertex,
  density = density
)

In [107]:
#generate an interval instance
interval = generate_interval()

In [108]:
#generate a random path
path = generate_random_path(nb_vertex_chosen)

In [111]:
# generate the neighbours of a path
neighbours = generate_neighbours(path, complete_graph, interval, 18)

In [114]:
# print("graph")
# print(graph)
# print("complete_graph")
# print(complete_graph)
# print("all_paths")
# print(all_paths)
# print("random_vertex")
# print(random_vertex)
# print("interval")
# print(interval)

In [115]:
complete_path = get_complete_path(path, all_paths)

In [117]:
print(random_vertex)
print(path)
print(complete_path)

[61 84 35 57 39 86 51 13 64 96 40 98 72 56 12  2 67  0 38 14]
[15 10 19  5  3 11 16 17  2  9  6  7  1 12 18 13  0  8  4 14]
[2, 82, 47, 40, 14, 86, 22, 16, 57, 16, 51, 98, 38, 67, 61, 0, 14, 35, 96, 21, 91, 51, 98, 14, 13, 84, 7, 72, 38, 72, 56, 61, 44, 64, 77, 39, 34, 89, 12]
